# Stellar Classification

In this notebook, we investigate the Gaia dataset in order to explore the stellar classification via neural networks.

In [ ]:
# Libraries for plotting and manipulating data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

# Libraries for studying the graphs
import networkx as nx
import community as community_louvain
import community.community_louvain as community_louvain
from scipy.spatial import KDTree


#Libraries for implenting and training a neural network
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras.models import Sequential
from keras import optimizers

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Library for the creation of the tables
from tabulate import tabulate

# Set Global Configuration Options

It is nice to specify global configuration options at the start of the notebook, so tweaking certain parameters does not require much "scrolling" down the notebook.

In [ ]:
# set a seed
seed=42

def space():
  print('\n')

# Uploading Data

In [ ]:
#from google.colab import files
#uploaded = files.upload()

data = pd.read_csv('Data_Gaia_reduced.csv',nrows=32000) #132000

# Show the shapes of the dataset
data.shape

# Get column names
columns_list = data.columns.tolist()  # Extract column names and convert to a list

# Dataset Preparation




Before doing any kind of machine learning, it’s important to understand the nature of the data with one is dealing with. We check some information about the dataset, including how many missing values there are, the shape of the data, as well as the dtypes of each of the columns.

## Structure of the data


As always with new data, it is essential that we understand what the different columns means. Luckily, the dataset brief gives us some insight about what they mean:

- RA_ICRS: Right ascension in the ICRS (International Celestial Reference System) coordinate system.
- DE_ICRS: Declination in the ICRS coordinate system.
- Teff: Estimated effective temperature of the celestial object by Gaia in Kelvins.
- Dist: Distance to the celestial object: inverse of the parallax, in parsecs.
- Rad: Object radius estimate in terms of solar radius.
- Rad-Flame: Object radius estimate in terms of solar radius.
- Lum: Estimated object luminosity in terms of solar luminosity.
- Mass: Mass estimate in terms of solar mass.
- Age: Celestial object age in giga years.
- z: Redshift in km/s.

In [ ]:
# General informations on the dataset
data.info()

### Removing Trailing Whitespace for Stellar Classes
It appears that the stellar classes have a lot of trailing whitespaces, which might impact our readability. Let's remove these.

In [ ]:
# Remove null rows
data.dropna(inplace=True)

In [ ]:
# General informations on the dataset
data.info()

### Simple description

In [ ]:
# Sum up of statistics of the features
data.describe() #.to_latex()

In [ ]:
# Presents the first non-null rows
data.head()

# Data analysis

We explore the statistics and the behaviors of the data under consideration.

### Spatial distribution

In [ ]:
# 3D Scatter Plot
def plot_3d_distribution(data):
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(data['RA_ICRS'], data['DE_ICRS'], data['Dist'], c=data['Dist'], cmap='viridis', alpha=0.7)
    ax.set_title("3D Spatial Distribution of Stars")
    ax.set_xlabel("Right Ascension (RA)")
    ax.set_ylabel("Declination (Dec)")
    ax.set_zlabel("Distance (parsecs)")
    plt.show()


# 2D Scatter Plots
def plot_2d_projections(data):
    plt.figure(figsize=(12, 6))

    # RA vs Dec
    plt.subplot(1, 2, 1)
    plt.scatter(data['RA_ICRS'], data['DE_ICRS'], c=data['Dist'], cmap='viridis', alpha=0.7)
    plt.colorbar(label='Distance (parsecs)')
    plt.title("RA vs Dec")
    plt.xlabel("Right Ascension (RA)")
    plt.ylabel("Declination (Dec)")

    # RA vs Distance
    plt.subplot(1, 2, 2)
    plt.scatter(data['RA_ICRS'], data['Dist'], c=data['DE_ICRS'], cmap='viridis', alpha=0.7)
    plt.colorbar(label='Declination (Dec)')
    plt.title("RA vs Distance")
    plt.xlabel("Right Ascension (RA)")
    plt.ylabel("Distance (parsecs)")

    plt.tight_layout()
    plt.show()

# Plot the distributions
plot_3d_distribution(data)
space()
plot_2d_projections(data)

In [ ]:
# Check for clustering in Distance
plt.figure(figsize=(8, 6))
plt.hist(data['Dist'], bins=30, color='skyblue', edgecolor='black', alpha=0.7)
plt.title("Histogram of Distances")
plt.xlabel("Distance (parsecs)")
plt.ylabel("Frequency")
plt.show()

# Normalize RA, Dec, and Distance to the range [0, 1] for analysis
scaler = MinMaxScaler()
data_normalized = data[['RA_ICRS', 'DE_ICRS', 'Dist']].copy()
data_normalized[['RA_ICRS', 'DE_ICRS', 'Dist']] = scaler.fit_transform(data_normalized)

# Subsampling
# If clustering is evident in the Distance histogram, subsample the data
def subsample_data(data, distance_column, bins=5, sample_size=100):
    data['DistanceBin'] = pd.cut(data[distance_column], bins=bins)
    sampled_data = data.groupby('DistanceBin').apply(lambda x: x.sample(n=min(len(x), sample_size), random_state=42))
    return sampled_data.reset_index(drop=True)

# Subsampled data based on Distance
data_subsampled = subsample_data(data, 'Dist', bins=5, sample_size=100)

# Plot the 3D distribution of the subsampled data
plot_3d_distribution(data_subsampled)

### Generation of the histograms

In [ ]:
!pip install diptest

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.5/197.5 kB 6.1 MB/s eta 0:00:00


In [ ]:
#Useful library for studying the system
import seaborn as sns
from scipy.stats import norm
from scipy.stats import gaussian_kde
from diptest import diptest  # Install using !pip install diptest
from scipy.stats import kurtosis, skew


In [ ]:
# It creates a histogram with a (optional) Gaussian on it
def hist_with_gaussian(df, column, bins, color_graph, Gaussian, subject):
    # Generation of the histogram:
    count, bins, ignored = plt.hist(df[column], bins, density=True, alpha=0.6, color=color_graph, label=f'Histogram of the {subject}')

    # Evaluation of the mean (mu) and standard deviation (std) for the data
    mu, std = norm.fit(df[column])

    xmin, xmax = plt.xlim()  # To get fixed x-axis limits for the distribution
    x = np.linspace(xmin, xmax, 15000)
    # Gaussian distribution
    p = norm.pdf(x, mu, std)

    # Plot
    if Gaussian:
        sums, bins = np.histogram(df[column].values, bins=bins, density=True, range=[xmin, xmax])
        bin_centers = (bins[:-1] + bins[1:]) / 2
        expected = (1 / np.sqrt(2 * np.pi * std ** 2)) * np.exp(-(bin_centers - mu) ** 2 / (2 * std ** 2))

        plt.plot(bin_centers, expected, 'o', color='blue', label='Expected bins center')
        plt.plot(bin_centers, sums, 'o', color='green', label='Predicted bins center')
        plt.plot(x, p, 'k', linewidth=2, color='#E76F51', label="Gaussian")
        plt.title(f"Histogram of the {subject}; result: {round(mu, 3)} ± {round(std, 3)}")
        plt.ylabel("Frequency")
        plt.xlabel(column)

    # Compute kurtosis and skewness
    data_kurtosis = kurtosis(df[column].dropna(), fisher=True)  # Fisher=True for excess kurtosis
    data_skewness = skew(df[column].dropna())

    # Return the computed mean, std, kurtosis, and skewness
    return mu, std, data_kurtosis, data_skewness

# It prints the data of the Gaussian
def print_values(data_print):
    print('\n')
    print('#### SUMMARY OF THE DATA: ####')
    headers = ['Subject', 'Mean value', 'Std Dev', 'Kurtosis', 'Skewness']  # Adding headers for kurtosis and skewness
    print(tabulate(data_print, headers=headers, tablefmt='latex', colalign=("center", "center", "center", "center", "center")))
    print('############################################')


In [ ]:
 ####################     Statistical Measures #########################
''' Test for bimodality using methods like the Hartigan's Dip Test or visual inspection of KDE plots.'''

# Visual Inspection: KDE Plots
def plot_kde_with_histogram(data, column):

    plt.figure(figsize=(10, 6))

    # Plot KDE and histogram
    sns.kdeplot(data[column], color='#2A9D8F', label='KDE')
    sns.histplot(data[column], bins=30, color='#F4A261', edgecolor=None, label=f'Histogram of the {column}', stat="density")


    # Fit a normal distribution to the data
    mu, std = norm.fit(data[column])

    # Generate x-values for the Gaussian curve
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)

    # Calculate the Gaussian curve
    p = norm.pdf(x, mu, std)

    '''
    sums, bins = np.histogram(data[column].values, bins=30, density=True, range=[xmin, xmax])
    bin_centers = (bins[:-1] + bins[1:]) / 2
    expected = (1 / np.sqrt(2 * np.pi * std ** 2)) * np.exp(-(bin_centers - mu) ** 2 / (2 * std ** 2))

    plt.plot(bin_centers, expected, 'o', color='blue', label='Expected bins center')
    plt.plot(bin_centers, sums, 'o', color='green', label='Predicted bins center')
    '''

    # Plot the Gaussian curve
    plt.plot(x, p, 'k', linewidth=2, color='#E76F51', label='Gaussian')
    plt.vlines(mu, 0, plt.ylim()[1], label="mean", color='#264653', ls='--', lw=1.4)


    plt.title(f'KDE, histogram and gaussian for {column}')
    plt.xlabel(column)
    plt.ylabel('Density')
    plt.legend()
    plt.show()


# Hartigan's Dip Test for Bimodality
def test_bimodality(data, column):
    values = data[column].dropna().values
    dip_stat, p_value = diptest(values)
    return dip_stat, p_value


In [ ]:
#### Implementation of the function and plot ####

# Implementation of the function and plot
data_print = []  # Initialize an empty list to store data
for column in columns_list:
    # Evaluation of the mean, std, kurtosis, and skewness
    mu, std, kurt, skewness = hist_with_gaussian(data, column, bins=20, color_graph='#F4A261', Gaussian=True, subject=column)

    # Plot annotation
    plt.vlines(mu, 0, plt.ylim()[1], label="mean", color='#264653', ls='--', lw=1.4)

    # Full plot configuration and show
    plt.title(f"Histogram of {column}")
    plt.tight_layout()
    plt.legend(loc="best")

    plt.show()

    data_print.append([column, mu, std, kurt, skewness])  # Append data to the list

    print('##############################################################')
    print('\n')


In [ ]:
# Show the relevant characteristics of the histograms
print_values(data_print)

In [ ]:
# Example: Visualize all numeric columns
numeric_columns = data.select_dtypes(include=[np.number]).columns
for column in numeric_columns:
    plot_kde_with_histogram(data, column)


# Test for bimodality in all numeric columns
print("\nHartigan's Dip Test Results:")
for column in numeric_columns:
    dip_stat, p_value = test_bimodality(data, column)
    print(f"Column: {column} | Dip Statistic: {dip_stat:.4f} | p-value: {p_value:.4e}")



In [ ]:
# Plot original and log-transformed Luminosity
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
sns.histplot(data['Lum'], bins=50, kde=True, ax=axes[0])
axes[0].set_title("Original Luminosity Distribution")

sns.histplot(data['Lum_log'], bins=50, kde=True, ax=axes[1])
axes[1].set_title("Log-Transformed Luminosity Distribution")

plt.show()


In [ ]:
# Create the histogram using seaborn's histplot: Mass vs Age
sns.histplot(data=data, x='Mass', y='Age', bins=30, cbar=True)

plt.title("Distribution of 'Age' with Respect to 'Mass'")
plt.xlabel('Mass [Solar Mass]')
plt.ylabel('Age [Gyr]')
plt.grid(True, linestyle='--', alpha=0.7)

plt.show()

In [ ]:
# Create the histogram using seaborn's histplot: Luminosity vs Age
sns.histplot(data=data, x='Lum', y='Age', bins=30, cbar=True)

plt.title("Distribution of 'Age' with Respect to 'Lum'")
plt.ylabel('Age [Gyr]')
plt.xlabel('Luminosity (Lum) [Solar Units]')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Create the histogram using seaborn's histplot: Redshift vs Age
sns.histplot(data=data, x='z', y='Age', bins=30, cbar=True)

plt.title("Distribution of 'Age' with Respect to 'z'")
plt.ylabel('Age [Gyr]')
plt.xlabel('Redshift (z) [km/s]')

plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Create the histogram using seaborn's histplot: Effective Temperature vs Age
sns.histplot(data=data, x='Teff', y='Age', bins=30, cbar=True)

plt.title("Distribution of 'Age' with Respect to 'Teff'")
plt.xlabel('Effective Temperature (Teff) [K]')
plt.ylabel('Age [Gyr]')

plt.grid(True, linestyle='--', alpha=0.7)
plt.show()



In [ ]:
# Create the histogram using seaborn's histplot: Radius vs Age
sns.histplot(data=data, x='Rad-Flame', y='Age', bins=30, cbar=True)

plt.title("Distribution of 'Age' with Respect to 'Rad-Flame'")
plt.xlabel('Radius [Solar Radius]')
plt.ylabel('Age [Gyr]')

plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Create the histogram using seaborn's histplot: log(g) vs Age
sns.histplot(data=data, x='logg', y='Age', bins=30, cbar=True)

plt.title("Distribution of 'Age' with Respect to 'log(g)'")
plt.xlabel('log(g)')
plt.ylabel('Age [Gyr]')

plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Create the histogram using seaborn's histplot: [Fe/H] vs Age
sns.histplot(data=data, x='[Fe/H]', y='Age', bins=30, cbar=True)

plt.title("Distribution of 'Age' with Respect to '[Fe/H]'")
plt.xlabel('[Fe/H]')
plt.ylabel('Age [Gyr]')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

### Correlation Matrix

We can investigate the correlation matrix. A correlation matrix is a statistical technique used to evaluate the relationship between two variables in a data set. The matrix is a table in which every cell contains a correlation coefficient, where 1 is considered a strong relationship between variables, 0 a neutral relationship and -1 a not strong relationship.

In [ ]:
#creation of the correlation matrix
data.corr(method='pearson')

#Plotting of the figure
plt.figure(figsize=(10,10))
plt.title("Correlation Matrix of the Dataset")
sns.heatmap(data.corr(method='pearson'), annot=True, cmap='coolwarm')

plt.show()


### Box plot

In [ ]:
############## box plot ##################

# Box Plot Visualization
def plot_boxplots(data, numerical_columns=None):
    """
    Plot box plots for numerical features in the dataset.

    Parameters:
    - data: pandas DataFrame, the dataset
    - numerical_columns: list of columns to include (default is all numeric columns)
    """
    if numerical_columns is None:
        # Automatically detect numeric columns
        numerical_columns = data.select_dtypes(include=['number']).columns

    # Plot each numeric feature in a box plot
    for column in numerical_columns:
        plt.figure(figsize=(10, 6))
        sns.boxplot(x=data[column], color='skyblue', flierprops={'markerfacecolor': 'red', 'marker': 'o'})
        plt.title(f'Box Plot of {column}')
        plt.xlabel(column)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()
        space()

# Automatically detect numeric columns
numerical_features = data.select_dtypes(include=['number']).columns

# Plot box plots for all numeric features
plot_boxplots(data, numerical_columns=numerical_features)

# Construction and plotting of the graph

We now present the graph study over 4 features: distance, luminosity, age and mass. These quatities permits to go into details about charachteristics of the stellar dataset.

Part of the code is blocked since the related analysis does not give any relevant information in our case, but could be useful in others.

In [ ]:
# Additional useful libraries for the graph analysis
from joblib import Parallel, delayed
from networkx.algorithms.community import modularity
from networkx.algorithms.community import louvain_communities
from scipy.stats import spearmanr
from scipy.spatial import KDTree

In [ ]:
#### Definition of the functions ####

# Function for plottting the degree of distribution
def plot_degree_distribution(graph, title):
  degree_sequence = [d for n, d in graph.degree()]

  # Calcola il numero di bin usando la regola di Freedman-Diaconis
  q1 = np.percentile(degree_sequence, 25)  # Primo quartile
  q3 = np.percentile(degree_sequence, 75)  # Terzo quartile
  iqr = q3 - q1  # Intervallo interquartile

  bin_width = 2 * iqr / len(degree_sequence)**(1/3)  # Larghezza del bin secondo Freedman-Diaconis
  bin_count = int((max(degree_sequence) - min(degree_sequence)) / bin_width) if bin_width > 0 else 10

  plt.hist(degree_sequence, bins=10, color='orange', edgecolor='black', density=True)
  plt.title(title)
  plt.xlabel('degree_sequence')
  plt.ylabel('Frequency')
  plt.show()

# Function for plottting the degree of centrality
def plot_degree_centrality(graph, title):
  degree_centrality = nx.degree_centrality(graph)
  plt.figure(figsize=(6, 5))
  nodes = nx.draw(graph, node_color=list(degree_centrality.values()),
                  node_size=100, cmap=plt.cm.plasma, edge_color='gray')
  plt.title(title)
  plt.show()

# Function for plottting the clustering of louvain
def plot_louvain_clusters(graph, title):
  partition = community_louvain.best_partition(graph)
  pos = nx.spring_layout(graph)
  plt.figure(figsize=(6, 5))
  nx.draw(graph, pos, node_color=list(partition.values()),
          node_size=50, cmap=plt.cm.jet, with_labels=False)
  plt.title(title)
  plt.show()


# Analyzes a list of graphs based on thresholds and star features
def analyze_graphs(graphs, thresholds, data):

    for idx, G in enumerate(graphs):
        print(f"Analysis for Graph with Threshold {thresholds[idx]:.2f}")
        print("=" * 50)

        # Compute and print graph density
        density = nx.density(G)
        print(f"Graph Density: {density:.6f}")

        # Compute and print average degree centrality
        degree_centrality = nx.degree_centrality(G)
        avg_centrality = sum(degree_centrality.values()) / len(degree_centrality)
        print(f"Average Degree Centrality: {avg_centrality:.4f}")

        # Identify the node with the highest degree centrality
        top_node = max(degree_centrality, key=degree_centrality.get)
        print(f"Top Node (ID: {top_node}) Degree Centrality: {degree_centrality[top_node]:.4f}")

        '''
        nodes = list(G.nodes())
        clustering_coefficients = Parallel(n_jobs=-1)(
            delayed(compute_clustering)(node, G) for node in nodes
        )
        avg_clustering = sum(clustering_coefficients) / len(clustering_coefficients)
        return avg_clustering
        print(f"Average Clustering Coefficient: {avg_clustering:.4f}")
        '''

        print("\n" + "=" * 50 + "\n")

# Analyzes a list of graphs, looking into clusters and stellar properties
def analyze_clusters_and_stellar_properties(graphs, thresholds, data):

    for idx, G in enumerate(graphs):
        print(f"\nAnalysis for Graph with Threshold {thresholds[idx]:.2f}")
        print("=" * 50)

        # Quantitative Analysis of Clusters
        # Use Louvain community detection
        communities = louvain_communities(G)
        num_clusters = len(communities)
        print(f"Number of Clusters: {num_clusters}")

        # Compute modularity score
        community_map = {node: cid for cid, community in enumerate(communities) for node in community}
        mod_score = modularity(G, communities)
        print(f"Modularity Score: {mod_score:.4f}")

        # Assign cluster labels to nodes
        nx.set_node_attributes(G, community_map, "cluster")

        '''
        # Compare with Stellar Properties
        # Extract cluster labels and corresponding properties
        clusters = nx.get_node_attributes(G, "cluster")
        degrees = dict(G.degree())

        # Prepare a DataFrame for analysis
        features = pd.DataFrame(data)  # Ensure 'data' is a DataFrame
        features["Cluster"] = features.index.map(clusters)
        features["Degree"] = features.index.map(degrees)

        # Correlate cluster labels with mass, age, and luminosity
        for prop in ["Mass", "Age", "Lum"]:
            if prop in features.columns:
                corr, pval = spearmanr(features["Cluster"], features[prop], nan_policy='omit')
                print(f"Spearman Correlation (Cluster vs {prop}): {corr:.4f} (p={pval:.4e})")

        # Create a 2x2 grid of subplots
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        axes = axes.flatten()  # Flatten axes for easier iteration

        # Scatter plot: Degree vs Mass colored by cluster
        scatter = axes[0].scatter(
            features["Mass"], features["Degree"], c=features["Cluster"], cmap="viridis", alpha=0.7, edgecolor="k"
        )
        axes[0].set_title(f"Mass vs Degree (Threshold: {thresholds[idx]:.2f})")
        axes[0].set_xlabel("Mass")
        axes[0].set_ylabel("Degree")
        axes[0].grid()
        plt.colorbar(scatter, ax=axes[0], label="Cluster")

        # Scatter plot: Luminosity vs Degree colored by cluster
        scatter = axes[1].scatter(
            features["Lum"], features["Degree"], c=features["Cluster"], cmap="plasma", alpha=0.7, edgecolor="k"
        )
        axes[1].set_title(f"Luminosity vs Degree (Threshold: {thresholds[idx]:.2f})")
        axes[1].set_xlabel("Luminosity")
        axes[1].set_ylabel("Degree")
        axes[1].grid()
        plt.colorbar(scatter, ax=axes[1], label="Cluster")

        # Scatter plot: [Fe/H] vs Degree colored by cluster
        scatter = axes[2].scatter(
            features["[Fe/H]"], features["Degree"], c=features["Cluster"], cmap="plasma", alpha=0.7, edgecolor="k"
        )
        axes[2].set_title(f"[Fe/H] vs Degree (Threshold: {thresholds[idx]:.2f})")
        axes[2].set_xlabel("[Fe/H]")
        axes[2].set_ylabel("Degree")
        axes[2].grid()
        plt.colorbar(scatter, ax=axes[2], label="Cluster")

        # Scatter plot: Rad-Flame vs Degree colored by cluster
        scatter = axes[3].scatter(
            features["Rad-Flame"], features["Degree"], c=features["Cluster"], cmap="plasma", alpha=0.7, edgecolor="k"
        )
        axes[3].set_title(f"Rad-Flame vs Degree (Threshold: {thresholds[idx]:.2f})")
        axes[3].set_xlabel("Rad-Flame")
        axes[3].set_ylabel("Degree")
        axes[3].grid()
        plt.colorbar(scatter, ax=axes[3], label="Cluster")

        # Adjust layout
        plt.tight_layout()
        plt.show()
        '''
        print("=" * 50)

# Function to compute edge weights based on different methods
def compute_weight(method, lum_diff, mass_diff, age_diff, lum_scale, mass_scale, age_scale):
    """Computes edge weight based on chosen method."""
    if method == "exponential":
        return np.exp(-((lum_diff / lum_scale)**2 + (mass_diff / mass_scale)**2 + (age_diff / age_scale)**2))

    elif method == "inverse_euclidean":
        return 1 / (1 + np.sqrt(lum_diff**2 + mass_diff**2 + age_diff**2))

    elif method == "cosine_similarity":
        vec1 = np.array([lum_diff, mass_diff, age_diff])
        vec2 = np.array([lum_scale, mass_scale, age_scale])
        return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2) + 1e-6)  # Avoid division by zero

    elif method == "manhattan":
        return 1 - ((lum_diff / lum_scale) + (mass_diff / mass_scale) + (age_diff / age_scale))

    else:
        raise ValueError("Unknown weight method!")

# function to visualized the graoh in different ways
def visualize_graph_layouts(graph):
    # Chosen layout
    layouts = {
        "Spring Layout": nx.spring_layout,
        "Circular Layout": nx.circular_layout,
        "Random Layout": nx.random_layout,
        "Shell Layout": nx.shell_layout,
        "Kamada-Kawai Layout": nx.kamada_kawai_layout,
        "Spectral Layout": nx.spectral_layout
    }

    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()

    # Show all the layouts
    for ax, (layout_name, layout_func) in zip(axes, layouts.items()):
        pos = layout_func(graph)  # Evaluate node's positions
        nx.draw(
            graph,
            pos,
            ax=ax,
            with_labels=False,
            node_size=80,
            node_color='deepskyblue',
            edge_color='gray',
            alpha=0.7
        )
        ax.set_title(layout_name, fontsize=14)

    plt.tight_layout()
    plt.show()





### All together

#### All together: no weight

In [ ]:
# Define thresholds
distance_thresholds = [500, 1500, 3500, 5000]  # parsecs
luminosity_thresholds = [5, 50, 150, 500]  # Log-scaled solar units
mass_thresholds = [2.0, 2.5, 3.0, 3.5]  # Focused on mean  1 std dev
age_thresholds = [0.2, 0.3, 0.4, 0.5]  # Focused around 1 std dev

# Create a KDTree for efficient nearest neighbor search based on spatial distances
kdtree = KDTree(data[['RA_ICRS', 'DE_ICRS', 'Dist']].values)

# Create a figure with 2x2 subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()  # Flatten the axes array for easy iteration

# Generate the graphs based on distance, luminosity, mass, and age thresholds
distance_graphs = []
for idx, threshold in enumerate(distance_thresholds):
    G_distance = nx.Graph()

    # Adding nodes (stars with attributes like Age, Distance, Luminosity, Mass)
    for index, row in data.iterrows():
        G_distance.add_node(
            index,
            Dist=row['Dist'],  # Primary feature
            RA=row['RA_ICRS'],  # Right Ascension
            DE=row['DE_ICRS'],  # Declination
            Lum=row['Lum'],  # Luminosity
            Mass=row['Mass'],  # Mass
            Age=row['Age']  # Age
        )

    # Finding neighbors within the threshold using KDTree
    for i in range(len(data)):
        indices = kdtree.query_ball_point(data.iloc[i][['RA_ICRS', 'DE_ICRS', 'Dist']].values, threshold)
        for j in indices:
            if i < j:  # Avoid duplicate edges
                lum_diff = abs(data.iloc[i]['Lum'] - data.iloc[j]['Lum'])
                mass_diff = abs(data.iloc[i]['Mass'] - data.iloc[j]['Mass'])
                age_diff = abs(data.iloc[i]['Age'] - data.iloc[j]['Age'])

                # Condition: distance, luminosity, mass, and age differences within thresholds
                if (
                    lum_diff < luminosity_thresholds[idx % len(luminosity_thresholds)] and
                    mass_diff < mass_thresholds[idx % len(mass_thresholds)] and
                    age_diff < age_thresholds[idx % len(age_thresholds)]
                ):
                    G_distance.add_edge(i, j)

    # Append the graph to the list
    distance_graphs.append(G_distance)

    # Visualizing the graph on the corresponding subplot
    nx.draw(
        G_distance,
        ax=axes[idx],
        with_labels=False,
        node_size=20,
        node_color='deepskyblue',
        edge_color='gray',
        alpha=0.7
    )
    axes[idx].set_title(f"Threshold: {round(threshold, 2)} parsec, Lum: {luminosity_thresholds[idx % len(luminosity_thresholds)]}, "
                        f"Mass: {mass_thresholds[idx % len(mass_thresholds)]}, Age: {age_thresholds[idx % len(age_thresholds)]}", fontsize=10)

plt.tight_layout()
plt.show()


#### All together: Weighted graph (Exponential weight chosen)

In [ ]:
# Define the weight method to use: 'exponential', 'inverse_euclidean', 'cosine_similarity', or 'manhattan'
weight_method = "exponential"

# Define thresholds
distance_thresholds = [500, 1500, 3500, 5000]  # parsecs
luminosity_thresholds = [5, 50, 150, 500]  # Log-scaled solar units
mass_thresholds = [2.0, 2.5, 3.0, 3.5]  # Focused on mean  1 std dev
age_thresholds = [0.2, 0.3, 0.4, 0.5]  # Focused around 1 std dev

# Create a KDTree for efficient nearest neighbor search based on spatial distances
kdtree = KDTree(data[['RA_ICRS', 'DE_ICRS', 'Dist']].values)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()  # Flatten the axes array for easy iteration

# Generate the graphs based on distance, luminosity, mass, and age thresholds
distance_graphs = []
for idx, threshold in enumerate(distance_thresholds):
    G_distance = nx.Graph()

    # Adding nodes (stars with attributes like Age, Distance, Luminosity, Mass)
    for index, row in data.iterrows():
        G_distance.add_node(
            index,
            Dist=row['Dist'],
            RA=row['RA_ICRS'],
            DE=row['DE_ICRS'],
            Lum=row['Lum'],
            Mass=row['Mass'],
            Age=row['Age']
        )

    # Finding neighbors within the threshold using KDTree
    for i in range(len(data)):
        indices = kdtree.query_ball_point(data.iloc[i][['RA_ICRS', 'DE_ICRS', 'Dist']].values, threshold)
        for j in indices:
            if i < j:  # Avoid duplicate edges
                lum_diff = abs(data.iloc[i]['Lum'] - data.iloc[j]['Lum'])
                mass_diff = abs(data.iloc[i]['Mass'] - data.iloc[j]['Mass'])
                age_diff = abs(data.iloc[i]['Age'] - data.iloc[j]['Age'])

                # Compute weight based on selected method
                weight = compute_weight(weight_method, lum_diff, mass_diff, age_diff,
                                        luminosity_thresholds[-1], mass_thresholds[-1], age_thresholds[-1])

                # Condition: distance, luminosity, mass, and age within thresholds
                if (
                    lum_diff < luminosity_thresholds[idx % len(luminosity_thresholds)] and
                    mass_diff < mass_thresholds[idx % len(mass_thresholds)] and
                    age_diff < age_thresholds[idx % len(age_thresholds)]
                ):
                    G_distance.add_edge(i, j, weight=weight)  # Store weight in the edge

    # Append the graph to the list
    distance_graphs.append(G_distance)

    # Extract edge weights for visualization
    edges, weights = zip(*nx.get_edge_attributes(G_distance, 'weight').items()) if len(G_distance.edges) > 0 else ([], [])

    # Visualizing the graph on the corresponding subplot
    pos = nx.spring_layout(G_distance, seed=42)  # Use force-directed layout
    nx.draw(
        G_distance,
        pos,
        ax=axes[idx],
        with_labels=False,
        node_size=20,
        node_color='deepskyblue',
        edge_color=weights,
        edge_cmap=plt.cm.Blues,
        alpha=0.7
    )
    axes[idx].set_title(f"Thresholds: {round(threshold, 2)} parsec, {luminosity_thresholds[idx % len(luminosity_thresholds)]} s.u., "
                        f" {mass_thresholds[idx % len(mass_thresholds)]}s.m.,{age_thresholds[idx % len(age_thresholds)]} Gy", fontsize=13)

plt.tight_layout()
plt.show()


#### All together: generic layout

In [ ]:
visualize_graph_layouts(G_distance)

#### All together: mixed spring and spectral

In [ ]:
# Step 1: Spectral Layout for Large-Scale Structure
pos_spectral = nx.spectral_layout(G_distance)

# Step 2: Spring Layout for Fine-Grained Adjustments
pos_refined = nx.spring_layout(G_distance, pos=pos_spectral, seed=42)

# Plot Final Network
plt.figure(figsize=(8, 8))
nx.draw(G_distance, pos_refined, node_size=5, edge_color='gray', alpha=0.5)
plt.title("Refined Galactic Structure Layout (Spectral + Spring)")
plt.show()

#### Analysis graph and clusters: All together

In [ ]:
# Function to evaluate and print: graph density, average degree centrality, top node ID and its degree of centrality
analyze_graphs(distance_graphs, distance_thresholds, data)

In [ ]:
# Function to evaluate and print the cluster's properties
analyze_clusters_and_stellar_properties(distance_graphs, distance_thresholds, data)

#### Plot of degree of distribution, centrality and louvain clustering for all the features

In [ ]:
# Applying the analysis functions to all four graphs: Degree of Distribution
for idx, graph in enumerate(distance_graphs):
    title = f"{round(distance_thresholds[idx], 2)} parsec, {luminosity_thresholds[idx % len(luminosity_thresholds)]} s.u., {mass_thresholds[idx % len(mass_thresholds)]}s.m., {age_thresholds[idx % len(age_thresholds)]} Gy"

    # Degree Distribution
    plot_degree_distribution(graph, f"Degree Distribution: {title}")
    print('\n')

In [ ]:
# Applying the analysis functions to all four graphs: Degree of Centrality
for idx, graph in enumerate(distance_graphs):
    title = f"{round(distance_thresholds[idx], 2)} parsec, {luminosity_thresholds[idx % len(luminosity_thresholds)]} s.u., {mass_thresholds[idx % len(mass_thresholds)]}s.m., {age_thresholds[idx % len(age_thresholds)]} Gy"

    plot_degree_centrality(graph, f"Degree Centrality: {title}")
    print('\n')

In [ ]:
# Applying the analysis functions to all four graphs: Louvain Clustering
for idx, graph in enumerate(distance_graphs):
    title = f"{round(distance_thresholds[idx], 2)} parsec, {luminosity_thresholds[idx % len(luminosity_thresholds)]} s.u., {mass_thresholds[idx % len(mass_thresholds)]}s.m., {age_thresholds[idx % len(age_thresholds)]} Gy"

    # Louvain Clustering
    plot_louvain_clusters(graph, f"Louvain Clusters: {title}")
    print('\n')

### Distance and luminosity together

In [ ]:
# Function to compute inverse Euclidean weight
def inverse_euclidean_weight(dist_diff, lum_diff):
    return 1 / (1 + np.sqrt(dist_diff**2 + lum_diff**2))  # Normalize weight

In [ ]:
distance_thresholds = [500, 1500, 3500, 5000]  # Parsecs
luminosity_thresholds = [5, 50, 150, 500]  # Solar luminosities

# Create a KDTree for efficient nearest neighbor search based on spatial distances
kdtree = KDTree(data[['RA_ICRS', 'DE_ICRS', 'Dist']].values)

# **Create a figure and axes for subplots**
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()  # Flatten the axes array for easy iteration

# Generate the graphs based on distance and luminosity thresholds
distance_graphs = []
for idx, threshold in enumerate(distance_thresholds):
    G_distance = nx.Graph()

    # Adding nodes (stars with attributes: Distance, Luminosity)
    for index, row in data.iterrows():
        G_distance.add_node(
            index,
            Dist=row['Dist'],  # Distance
            Lum=row['Lum'],  # Luminosity
        )

    # Finding neighbors within the threshold using KDTree
    for i in range(len(data)):
        indices = kdtree.query_ball_point(data.iloc[i][['RA_ICRS', 'DE_ICRS', 'Dist']].values, threshold)
        for j in indices:
            if i < j:  # Avoid duplicate edges
                dist_diff = abs(data.iloc[i]['Dist'] - data.iloc[j]['Dist'])
                lum_diff = abs(data.iloc[i]['Lum'] - data.iloc[j]['Lum'])

                # Compute inverse Euclidean weight
                weight = inverse_euclidean_weight(dist_diff, lum_diff)

                # Condition: distance and luminosity within thresholds
                if (
                    lum_diff < luminosity_thresholds[idx % len(luminosity_thresholds)]
                ):
                    G_distance.add_edge(i, j, weight=weight)  # Store weight in the edge

    # Append the graph to the list
    distance_graphs.append(G_distance)

    # Extract edge weights for visualization
    edges, weights = zip(*nx.get_edge_attributes(G_distance, 'weight').items()) if len(G_distance.edges) > 0 else ([], [])

    # Visualizing the graph on the corresponding subplot
    pos = nx.spring_layout(G_distance, seed=42)  # Use force-directed layout
    nx.draw(
        G_distance,
        pos,
        ax=axes[idx],
        with_labels=False,
        node_size=20,
        node_color='deepskyblue',
        edge_color=weights,
        edge_cmap=plt.cm.Blues,  # Edge colors represent weight strength
        alpha=0.7
    )
    axes[idx].set_title(f"Thresholds: {round(threshold, 2)} parsec, {luminosity_thresholds[idx % len(luminosity_thresholds)]} s.u.",
                        fontsize=13)

plt.tight_layout()
plt.show()


#### Analysis graph and clusters: Distance and luminosity

In [ ]:
# Function to evaluate and print: graph density, average degree centrality, top node ID and its degree of centrality
analyze_graphs(distance_graphs, distance_thresholds, data)

In [ ]:
# Function to evaluate and print the cluster's properties
analyze_clusters_and_stellar_properties(distance_graphs, distance_thresholds, data)

#### Plot of degree of distribution, centrality and louvain clustering for luminosity and distance

In [ ]:
# Applying the analysis functions to all four graphs: Degree of Distribution
for idx, graph in enumerate(distance_graphs):
    title = f"{round(distance_thresholds[idx],2)} parsec and {round(luminosity_thresholds[idx],2)} s.u."

    # Degree Distribution
    plot_degree_distribution(graph, f"Degree of distribution: {title}")
    print('\n')

In [ ]:
# Applying the analysis functions to all four graphs: Degree of Centrality
for idx, graph in enumerate(distance_graphs):
    title = f"{round(distance_thresholds[idx],2)} parsec and {round(luminosity_thresholds[idx],2)} s.u."

    plot_degree_centrality(graph, f"Degree Centrality: {title}")
    print('\n')

In [ ]:
# Applying the analysis functions to all four graphs: Louvain Clustering
for idx, graph in enumerate(distance_graphs):
    title = f"{round(distance_thresholds[idx],2)} parsec and {round(luminosity_thresholds[idx],2)} s. u."

    # Louvain Clustering
    plot_louvain_clusters(graph, f"Louvain Clusters: {title}")
    print('\n')

## Age and mass together




### Age and mass together: Exp weight


In [ ]:
# Function to compute Exponential Similarity
def exponential_similarity(mass1, age1, mass2, age2, T_mass, T_age):
    """Computes the Exponential Similarity between two stars based on Mass and Age."""
    mass_diff = abs(mass1 - mass2) / T_mass
    age_diff = abs(age1 - age2) / T_age

    similarity = np.exp(- (mass_diff ** 2) - (age_diff ** 2))

    return similarity  # Higher means more similar

In [ ]:
# Define Mass and Age thresholds
mass_thresholds = [2.0, 2.5, 3.0, 3.5]  # Focused on mean  1 std dev
age_thresholds = [0.2, 0.3, 0.4, 0.5]  # Focused around 1 std dev

# Create a KDTree for efficient nearest neighbor search based on Mass and Age
kdtree = KDTree(data[['Age', 'Mass']].values)

# Create a figure and axes for subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()  # Flatten the axes array for easy iteration

# Generate the graphs based on Age and Mass thresholds
age_mass_graphs = []
for idx, threshold in enumerate(age_thresholds):  # Using age thresholds as reference
    G_age_mass = nx.Graph()

    # Adding nodes (stars with attributes: Age, Mass)
    for index, row in data.iterrows():
        G_age_mass.add_node(
            index,
            Mass=row['Mass'],
            Age=row['Age']
        )

    # Finding neighbors within the threshold using KDTree
    for i in range(len(data)):
        indices = kdtree.query_ball_point(data.iloc[i][['Age', 'Mass']].values, threshold)
        for j in indices:
            if i < j:  # Avoid duplicate edges
                mass1, age1 = data.iloc[i]['Mass'], data.iloc[i]['Age']
                mass2, age2 = data.iloc[j]['Mass'], data.iloc[j]['Age']

                # Compute Exponential Similarity
                weight = exponential_similarity(mass1, age1, mass2, age2,
                                                T_mass=mass_thresholds[-1],
                                                T_age=age_thresholds[-1])

                # Condition: age and mass within thresholds
                if (
                    abs(mass1 - mass2) < mass_thresholds[idx % len(mass_thresholds)] and
                    abs(age1 - age2) < age_thresholds[idx % len(age_thresholds)]
                ):
                    G_age_mass.add_edge(i, j, weight=weight)  # Store weight in the edge

    # Append the graph to the list
    age_mass_graphs.append(G_age_mass)

    # Extract edge weights for visualization
    edges, weights = zip(*nx.get_edge_attributes(G_age_mass, 'weight').items()) if len(G_age_mass.edges) > 0 else ([], [])

    # Visualizing the graph on the corresponding subplot
    pos = nx.spring_layout(G_age_mass, seed=42)  # Use force-directed layout
    nx.draw(
        G_age_mass,
        pos,
        ax=axes[idx],
        with_labels=False,
        node_size=20,
        node_color='deepskyblue',
        edge_color=weights,
        edge_cmap=plt.cm.Blues,  # Edge colors represent similarity strength
        alpha=0.7
    )
    axes[idx].set_title(f"Thresholds: {round(threshold, 2)} Gy, {mass_thresholds[idx % len(mass_thresholds)]} s.m.",fontsize=13)

plt.tight_layout()
plt.show()


### Age and mass together: Euclidean weight


In [ ]:
# Function to compute Inverse Euclidean Weight
def inverse_euclidean_weight(mass1, age1, mass2, age2):
    """Computes the Inverse Euclidean Weight between two stars based on Mass and Age."""
    mass_diff = mass1 - mass2
    age_diff = age1 - age2

    distance = np.sqrt(mass_diff**2 + age_diff**2)

    weight = 1 / (1 + distance)  # Normalize weight (closer values have higher weights)
    return weight

In [ ]:
# Define Mass and Age thresholds
mass_thresholds = [2.0, 2.5, 3.0, 3.5]  # Focused on mean  1 std dev
age_thresholds = [0.2, 0.3, 0.4, 0.5]  # Focused around 1 std dev

# Create a KDTree for efficient nearest neighbor search based on Mass and Age
kdtree = KDTree(data[['Age', 'Mass']].values)

# Create a figure and axes for subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()  # Flatten the axes array for easy iteration

# Generate the graphs based on Age and Mass thresholds
age_mass_graphs = []
for idx, threshold in enumerate(age_thresholds):  # Using age thresholds as reference
    G_age_mass = nx.Graph()

    # Adding nodes (stars with attributes: Age, Mass)
    for index, row in data.iterrows():
        G_age_mass.add_node(
            index,
            Mass=row['Mass'],
            Age=row['Age']
        )

    # Finding neighbors within the threshold using KDTree
    for i in range(len(data)):
        indices = kdtree.query_ball_point(data.iloc[i][['Age', 'Mass']].values, threshold)
        for j in indices:
            if i < j:  # Avoid duplicate edges
                mass1, age1 = data.iloc[i]['Mass'], data.iloc[i]['Age']
                mass2, age2 = data.iloc[j]['Mass'], data.iloc[j]['Age']

                # Compute Inverse Euclidean Weight
                weight = inverse_euclidean_weight(mass1, age1, mass2, age2)

                # Condition: age and mass within thresholds
                if (
                    abs(mass1 - mass2) < mass_thresholds[idx % len(mass_thresholds)] and
                    abs(age1 - age2) < age_thresholds[idx % len(age_thresholds)]
                ):
                    G_age_mass.add_edge(i, j, weight=weight)  # Store weight in the edge

    # Append the graph to the list
    age_mass_graphs.append(G_age_mass)

    # Extract edge weights for visualization
    edges, weights = zip(*nx.get_edge_attributes(G_age_mass, 'weight').items()) if len(G_age_mass.edges) > 0 else ([], [])

    # Visualizing the graph on the corresponding subplot
    pos = nx.spring_layout(G_age_mass, seed=42)  # Use force-directed layout
    nx.draw(
        G_age_mass,
        pos,
        ax=axes[idx],
        with_labels=False,
        node_size=20,
        node_color='deepskyblue',
        edge_color=weights,
        edge_cmap=plt.cm.Blues,  # Edge colors represent similarity strength
        alpha=0.7
    )
    axes[idx].set_title(f"Thresholds: {round(threshold, 2)} Gy, {mass_thresholds[idx % len(mass_thresholds)]} s.m.",fontsize=13)

plt.tight_layout()
plt.show()


### Age and mass together: Spectral layout and exp weight

In [ ]:
# Function to compute Exponential Similarity
def exponential_similarity(mass1, age1, mass2, age2, T_mass, T_age):
    """Computes the Exponential Similarity between two stars based on Mass and Age."""
    mass_diff = abs(mass1 - mass2) / T_mass
    age_diff = abs(age1 - age2) / T_age

    similarity = np.exp(- (mass_diff ** 2) - (age_diff ** 2))

    return similarity  # Higher means more similar

In [ ]:
# Define Mass and Age thresholds
mass_thresholds = [2.0, 2.5, 3.0, 3.5]  # Focused on mean  1 std dev
age_thresholds = [0.2, 0.3, 0.4, 0.5]  # Focused around 1 std dev

# Create a KDTree for efficient nearest neighbor search based on Mass and Age
kdtree = KDTree(data[['Age', 'Mass']].values)

# Create a figure and axes for subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()  # Flatten the axes array for easy iteration

# Generate the graphs based on Age and Mass thresholds
age_mass_graphs = []
for idx, threshold in enumerate(age_thresholds):  # Using age thresholds as reference
    G_age_mass = nx.Graph()

    # Adding nodes (stars with attributes: Age, Mass)
    for index, row in data.iterrows():
        G_age_mass.add_node(
            index,
            Mass=row['Mass'],
            Age=row['Age']
        )

    # Finding neighbors within the threshold using KDTree
    for i in range(len(data)):
        indices = kdtree.query_ball_point(data.iloc[i][['Age', 'Mass']].values, threshold)
        for j in indices:
            if i < j:  # Avoid duplicate edges
                mass1, age1 = data.iloc[i]['Mass'], data.iloc[i]['Age']
                mass2, age2 = data.iloc[j]['Mass'], data.iloc[j]['Age']

                # Compute Exponential Similarity
                weight = exponential_similarity(mass1, age1, mass2, age2,
                                                T_mass=mass_thresholds[-1],
                                                T_age=age_thresholds[-1])

                # Condition: age and mass within thresholds
                if (
                    abs(mass1 - mass2) < mass_thresholds[idx % len(mass_thresholds)] and
                    abs(age1 - age2) < age_thresholds[idx % len(age_thresholds)]
                ):
                    G_age_mass.add_edge(i, j, weight=weight)  # Store weight in the edge

    # Append the graph to the list
    age_mass_graphs.append(G_age_mass)

    # Extract edge weights for visualization
    edges, weights = zip(*nx.get_edge_attributes(G_age_mass, 'weight').items()) if len(G_age_mass.edges) > 0 else ([], [])

    # Visualizing the graph on the corresponding subplot
    pos = nx.spectral_layout(G_age_mass)  # Use force-directed layout
    nx.draw(
        G_age_mass,
        pos,
        ax=axes[idx],
        with_labels=False,
        node_size=20,
        node_color='deepskyblue',
        edge_color=weights,
        edge_cmap=plt.cm.Blues,  # Edge colors represent similarity strength
        alpha=0.7
    )
    axes[idx].set_title(f"Thresholds: {round(threshold, 2)} Gy, {mass_thresholds[idx % len(mass_thresholds)]} s.m.",fontsize=13)

plt.tight_layout()
plt.show()


### Analysis graph and clusters: Distance and luminosity

In [ ]:
# Function to evaluate and print: graph density, average degree centrality, top node ID and its degree of centrality
analyze_graphs(age_mass_graphs, age_thresholds, data)

In [ ]:
# Function to evaluate and print the cluster's properties
analyze_clusters_and_stellar_properties(age_mass_graphs, age_thresholds, data)

### Plot of degree of distribution, centrality and louvain clustering for luminosity and distance

In [ ]:
# Applying the analysis functions to all four graphs: Degree of Distribution
for idx, graph in enumerate(age_mass_graphs):
    title = f"{age_thresholds[idx % len(mass_thresholds)]} Gy, {mass_thresholds[idx % len(mass_thresholds)]} s.m."

    # Degree Distribution
    plot_degree_distribution(graph, f"Degree Distribution: {title}")
    print('\n')

In [ ]:
# Applying the analysis functions to all four graphs: Degree of Centrality
for idx, graph in enumerate(age_mass_graphs):
    title = f"{age_thresholds[idx % len(mass_thresholds)]} Gy, {mass_thresholds[idx % len(mass_thresholds)]} s.m."

    plot_degree_centrality(graph, f"Degree Centrality: {title}")
    print('\n')

In [ ]:
# Applying the analysis functions to all four graphs: Louvain Clustering
for idx, graph in enumerate(age_mass_graphs):
    title = f"{age_thresholds[idx % len(mass_thresholds)]} Gy, {mass_thresholds[idx % len(mass_thresholds)]} s.m."

    # Louvain Clustering
    plot_louvain_clusters(graph, f"Louvain Clusters: {title}")
    print('\n')

## Graph of the relative distance




We proceed to construct the graph of the relative distances. We will use "kdtree" to speed up the computation and use 4 meaningful thresholds: one smaller than the mean, the mean, one little higher than the mean and one greater than the mean. This permits us to have a full view on the dataset.

In [ ]:
#### Definition of the needed quantities ####

# Using only the filtered distances for KDTree
distances = data[['Dist']].values
kdtree = KDTree(distances)

# evaluate the mean distance
mean_distance = distances.mean()

# Define multiple distance thresholds for comparison
distance_thresholds = [0.5*mean_distance, mean_distance, (mean_distance +0.10*mean_distance), 2*mean_distance]
print(distance_thresholds)

[1765.2297997607654, 3530.459599521531, 3883.505559473684, 7060.919199043062]


In [ ]:
#### Graph construction ####

# Create a figure with 2x2 subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()  # Flatten the axes array for easy iteration

# Generate the graphs based on distance thresholds
distance_graphs = []
for idx, threshold in enumerate(distance_thresholds):
    G_distance = nx.Graph()

    # Adding nodes (stars with attributes like Age, distance, luminosity, Mass)
    for index, row in data.iterrows():
      G_distance.add_node(
          index,
          Dist=row['Dist'],        # Primary feature
          RA=row['RA_ICRS'],       # Right Ascension for spatial context
          DE=row['DE_ICRS'],       # Declination for spatial context
          Lum=row['Lum'],          # Luminosity for brightness context
          Mass=row['Mass'],        # Mass for understanding spatial clustering of massive stars
          Age=row['Age']           # Age for evolutionary insights: older stars may populate farther regions
      )

    # Finding neighbors within the threshold using KDTree
    for i in range(len(data)):
        indices = kdtree.query_ball_point(distances[i], threshold)
        for j in indices:
            if i < j:  # Avoid duplicate edges
                G_distance.add_edge(i, j)


    # Append the graph to the list
    distance_graphs.append(G_distance)

    #Visualizing the graph on the corresponding subplot
    nx.draw(
        G_distance,
        ax=axes[idx],
        with_labels=False,
        node_size=20,
        node_color='deepskyblue',
        edge_color='gray',
        alpha=0.7
    )
    axes[idx].set_title(f"Threshold: {round(threshold,2)} parsec", fontsize=10)

plt.tight_layout()
plt.show()


### Analysis graph and clusters: Distance

In [ ]:
# Function to evaluate and print: graph density, average degree centrality, top node ID and its degree of centrality
analyze_graphs(distance_graphs, distance_thresholds, data)

In [ ]:
# Function to evaluate and print the cluster's properties
analyze_clusters_and_stellar_properties(distance_graphs, distance_thresholds, data)

#### Plot of degree of distribution, centrality and louvain clustering for distance

In [ ]:
# Applying the analysis functions to all four graphs: Degree of Distribution
for idx, graph in enumerate(distance_graphs):
    title = f"Graph with Threshold {round(distance_thresholds[idx],2)} parsec"

    # Degree Distribution
    plot_degree_distribution(graph, f"Degree Distribution: {title}")
    print('\n')

In [ ]:
# Applying the analysis functions to all four graphs: Degree of Centrality
for idx, graph in enumerate(distance_graphs):
    title = f"Graph with Threshold {round(distance_thresholds[idx],2)} parsec"

    plot_degree_centrality(graph, f"Degree Centrality: {title}")
    print('\n')

In [ ]:
# Applying the analysis functions to all four graphs: Louvain Clustering
for idx, graph in enumerate(distance_graphs):
    title = f"Graph with Threshold {round(distance_thresholds[idx],2)} parsec"

    # Louvain Clustering
    plot_louvain_clusters(graph, f"Louvain Clusters: {title}")
    print('\n')

## Graph of the relative luminosities




In [ ]:
#### Definition of the needed quantities ####

luminosities = data[['Lum']].values
kdtree = KDTree(luminosities)

# evaluate the mean luminosity
mean_luminosity = luminosities.mean()

# Define multiple luminosity thresholds for comparison
luminosity_thresholds = [0.5*mean_luminosity,mean_luminosity, (mean_luminosity+0.10*mean_luminosity),2*mean_luminosity]
print(luminosity_thresholds)

[44.83322198066188, 89.66644396132376, 98.63308835745615, 179.33288792264753]


In [ ]:
#### Graph construction ####

# Create a figure with 2x2 subplots
fig, axes = plt.subplots(2,2, figsize=(12, 10))
axes = axes.flatten()  # Flatten the axes array for easy iteration

# Generate the graphs based on luminosity thresholds
luminosity_graphs = []
for idx,threshold in enumerate(luminosity_thresholds):
    G_luminosity = nx.Graph()

    # Adding nodes
    for index, row in data.iterrows():
        G_luminosity.add_node(
            index,
            Lum=row['Lum'],         # Primary feature
            Mass=row['Mass'],        # Stellar mass to explore its relationship with luminosity
            Teff=row['Teff'],        # Effective temperature for stellar classification
            Age=row['Age'],          # Age to analyze brightness evolution
            Rad=row['Rad-Flame']           # Larger stars emit more energy
        )

    # Adding edges using KDTree
    for i in range(len(data)):
        indices = kdtree.query_ball_point(luminosities[i], threshold)
        for j in indices:
            if i < j:
                G_luminosity.add_edge(i, j)

    luminosity_graphs.append(G_luminosity)

    # Visualizing the graph on the corresponding subplot
    nx.draw(
        G_luminosity,
        ax=axes[idx],
        with_labels=False,
        node_size=20,
        node_color='deepskyblue',
        edge_color='gray',
        alpha=0.7
    )
    axes[idx].set_title(f"Threshold: {round(threshold,2)} in solar luminosity ", fontsize=10)

plt.tight_layout()
plt.show()


#### Analysis graph and clusters: Luminosity

In [ ]:
# Function to evaluate and print: graph density, average degree centrality, top node ID and its degree of centrality
analyze_graphs(luminosity_graphs, luminosity_thresholds, data)


In [ ]:
# Function to evaluate and print the cluster's properties
analyze_clusters_and_stellar_properties( luminosity_graphs,  luminosity_thresholds, data)

#### Plot of degree of distribution, centrality and louvain clustering for Luminosity

In [ ]:
# Applying the analysis functions to all four graphs: Degree of Distribution
for idx, graph in enumerate(luminosity_graphs):
    title = f"Graph with Luminosity Threshold {round(luminosity_thresholds[idx],2)}"

    # Degree Distribution
    plot_degree_distribution(graph, f"Degree Distribution: {title}")
    print('\n')

In [ ]:
# Applying the analysis functions to all four graphs: Degree of Centrality
for idx, graph in enumerate(luminosity_graphs):
    title = f"Graph with Luminosity Threshold {round(luminosity_thresholds[idx],2)}"

    # Degree Centrality
    plot_degree_centrality(graph, f"Degree Centrality: {title}")
    print('\n')


In [ ]:
# Applying the analysis functions to all four graphs: Louvain Clustering
for idx, graph in enumerate(luminosity_graphs):
    title = f"Graph with Luminosity Threshold {round(luminosity_thresholds[idx],2)}"

    # Louvain Clustering
    plot_louvain_clusters(graph, f"Louvain Clusters: {title}")
    print('\n')

## Graph of the relative Ages



In [ ]:
#### Definition of the needed quantities ####

Ages = data[['Age']].values
kdtree = KDTree(Ages)

# evaluate the mean age
mean_age = Ages.mean()

# Define multiple Age thresholds for comparison
Age_thresholds = [0.5*mean_age, mean_age,(mean_age+0.10*mean_age), 2*mean_age]
print(Age_thresholds)

[0.14810147527910686, 0.2962029505582137, 0.3258232456140351, 0.5924059011164274]


In [ ]:
#### Graph construction ####

# Create a figure with 2x2 subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()  # Flatten the axes array for easy iteration

# Generate the graphs based on Age thresholds
Age_graphs = []
for idx, threshold in enumerate(Age_thresholds):
    G_Age = nx.Graph()

    # Adding nodes
    for index, row in data.iterrows():
        G_Age.add_node(
          index,
          Age=row['Age'],         # Primary feature
          Mass=row['Mass'],        # Massive stars evolve differently
          Lum=row['Lum'],          # Brightness for evolutionary comparison
          Metallicity=row['[Fe/H]'],  # To study star formation history
          Rad=row['Rad-Flame']           # Radius for evolutionary stage analysis
        )
    # Adding edges using KDTree
    for i in range(len(data)):
        indices = kdtree.query_ball_point(Ages[i], threshold)
        for j in indices:
            if i < j:
                G_Age.add_edge(i, j)

    Age_graphs.append(G_Age)

    # Visualizing the graph on the corresponding subplot
    nx.draw(
        G_Age,
        ax=axes[idx],
        with_labels=False,
        node_size=20,
        node_color='deepskyblue',
        edge_color='gray',
        alpha=0.7
    )
    axes[idx].set_title(f"Threshold: {round(threshold,2)}", fontsize=10)

plt.tight_layout()  # Adjust layout for better spacing
plt.show()


#### Analysis graph and clusters: Age

In [ ]:
# Function to evaluate and print: graph density, average degree centrality, top node ID and its degree of centrality
analyze_graphs(Age_graphs, Age_thresholds, data)


In [ ]:
# Function to evaluate and print the cluster's properties
analyze_clusters_and_stellar_properties(Age_graphs, Age_thresholds, data)

#### Plot of degree of distribution, centrality and louvain clustering for Age

In [ ]:
# Applying the analysis functions to all four graphs: Degree of Distribution
for idx, graph in enumerate(Age_graphs):
    title = f"Graph with Age Threshold {round(Age_thresholds[idx],2)}"

    # Degree Distribution
    plot_degree_distribution(graph, f"Degree Distribution: {title}")
    print('\n')

In [ ]:
# Applying the analysis functions to all four graphs: Degree of Centrality
for idx, graph in enumerate(Age_graphs):
    title = f"Graph with Age Threshold {round(Age_thresholds[idx],2)}"

    # Degree Centrality
    plot_degree_centrality(graph, f"Degree Centrality: {title}")
    print('\n')

In [ ]:
# Applying the analysis functions to all four graphs: Louvain Clustering
for idx, graph in enumerate(Age_graphs):
    title = f"Graph with Age Threshold {round(Age_thresholds[idx],2)}"


    # Louvain Clustering
    plot_louvain_clusters(graph, f"Louvain Clusters: {title}")
    print('\n')

## Graph of the relative Masses




In [ ]:
#### Definition of the needed quantities ####

Masses = data[['Mass']].values
kdtree = KDTree(Masses)

# evaluate the mean mass
mean_mass = Masses.mean()

# Define multiple mass thresholds for comparison
Mass_thresholds = [0.5*mean_mass, mean_mass, (mean_mass +0.10*mean_mass), 2* mean_mass]
print(Mass_thresholds)

[1.3628137958532696, 2.725627591706539, 2.998190350877193, 5.451255183413078]


In [ ]:
#### Graph construction ####

# Create a figure with 2x2 subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()  # Flatten the axes array for easy iteration

# Generate the graphs based on Mass thresholds
Mass_graphs = []
for idx,threshold in enumerate(Mass_thresholds):
    G_Mass = nx.Graph()

    # Adding nodes
    for index, row in data.iterrows():
        G_Mass.add_node(
        index,
        Mass=row['Mass'],        # Primary feature
        Lum=row['Lum'],          # Luminosity to examine massive stars' brightness
        Teff=row['Teff'],        # Effective temperature for classification
        Age=row['Age'],          # Tracks stellar evolution
        Rad=row['Rad-Flame']           # Stellar radius for physical properties
    )

    # Adding edges using KDTree
    for i in range(len(data)):
        indices = kdtree.query_ball_point(Masses[i], threshold)
        for j in indices:
            if i < j:
                G_Mass.add_edge(i, j)

    Mass_graphs.append(G_Mass)

    # Visualizing the graph on the corresponding subplot
    nx.draw(
        G_Mass,
        ax=axes[idx],
        with_labels=False,
        node_size=20,
        node_color='deepskyblue',
        edge_color='gray',
        alpha=0.7
    )
    axes[idx].set_title(f"Threshold: {round(threshold,2)}", fontsize=10)

plt.tight_layout()  # Adjust layout for better spacing
plt.show()


<Figure size 640x480 with 0 Axes>

#### Analysis graph and clusters: Mass

In [ ]:
# Function to evaluate and print: graph density, average degree centrality, top node ID and its degree of centrality
analyze_graphs(Mass_graphs, Mass_thresholds, data)


In [ ]:
# Function to evaluate and print the cluster's properties
analyze_clusters_and_stellar_properties(Mass_graphs, Mass_thresholds, data)

#### Plot of degree of distribution, centrality and louvain clustering for Mass

In [ ]:
# Applying the analysis functions to all four graphs: Degree of Distribution
for idx, graph in enumerate(Mass_graphs):
    title = f"Graph with Mass Threshold {round(Mass_thresholds[idx],2)}"

    # Degree Distribution
    plot_degree_distribution(graph, f"Degree Distribution: {title}")
    print('\n')

In [ ]:
# Applying the analysis functions to all four graphs: Degree of Centrality
for idx, graph in enumerate(Mass_graphs):
    title = f"Graph with Mass Threshold {round(Mass_thresholds[idx],2)}"

    # Degree Centrality
    plot_degree_centrality(graph, f"Degree Centrality: {title}")
    print('\n')

In [ ]:
# Applying the analysis functions to all four graphs: Louvain Clustering
for idx, graph in enumerate(Mass_graphs):
    title = f"Graph with Mass Threshold {round(Mass_thresholds[idx],2)}"

    # Louvain Clustering
    plot_louvain_clusters(graph, f"Louvain Clusters: {title}")
    print('\n')

# Introducing closeness and centrality for the training

***This part has not been implemented, but can be used for further developments of the analysis.***

This code calculates closeness centrality and degree centrality for each node (star) in your graph and adds them as new columns to your DataFrame, `data`.

- **Closeness centrality** measures the average distance of a node to all other nodes in the graph. Nodes with high closeness centrality are considered more "central" because they can reach other nodes quickly.

- **Degree centrality** measures the number of connections a node has. Nodes with high degree centrality are considered more "influential" because they have direct connections with many other nodes.

By adding these metrics as new columns to the DataFrame, you can use them as new features to train machine learning models and achieve more accurate predictions.

In [ ]:
# Calcola closeness centrality
closeness_centrality = nx.closeness_centrality(G)

# Calcola degree centrality
degree_centrality = nx.degree_centrality(G)

# Aggiungi le metriche come nuove colonne al dataframe
data['closeness_centrality'] = data['Unnamed: 0'].map(closeness_centrality)
data['degree_centrality'] = data['Unnamed: 0'].map(degree_centrality)

# Data Analysis

## Data splitting

Splitting a dataset into 20-80% helps evaluate model performance on unseen data, prevents overfitting, and aids in hyperparameter tuning and model selection.

In [ ]:
# Split arrays or matrices into random train and test subsets.
train,test=train_test_split(data, test_size=0.2, random_state=seed, shuffle=True)

In [ ]:
#plotting the diagrams
mu_tr, std_tr = hist_with_gaussian(train, 'Age',bins=30,color_graph='red',Gaussian=False,subject='train set')
mu_ts, std_ts = hist_with_gaussian(test, 'Age',bins=30, color_graph='blue',Gaussian=False,subject='test set')


plt.tight_layout()
plt.legend(loc="best")

plt.show()

The two datasets seem to be well balanced. Assuming the two distributions to be normal, they share values very similar both in mean value and standard deviation.



# Simple Perceptron

We design and implement a simple perceptron model to predict the Age of stars.

In [ ]:
# Additional library
from sklearn.preprocessing import StandardScaler

In [ ]:
#### Definition of the functions ####
# Function to prepare the data for the model
def preparing_data(train,test,scaled):
  # Select the only meaningful parameters that needs to be used
  if scaled==True:
    # function already implemented in the notebook: Only the relevant variables are taken into account
    X_train_scaled, X_test_scaled = scaled_train_test(train, test)
    X_train = X_train_scaled[ [ 'logg', '[Fe/H]','Teff', 'Rad-Flame','Lum' ,'Mass'] ].values
    y_train = X_train_scaled['Age'].values
    X_test = X_test_scaled[ [ 'logg', '[Fe/H]', 'Teff','Rad-Flame','Lum', 'Mass'] ].values
    y_test = X_test_scaled['Age'].values

  else:
    X_train = train[[ 'logg', '[Fe/H]', 'Teff','Rad-Flame','Lum','Mass']].values
    y_train = train['Age'].values
    X_test = test[[ 'logg', '[Fe/H]','Teff', 'Rad-Flame','Lum','Mass']].values
    y_test = test['Age'].values

  return X_train.shape[1], X_train, y_train, X_test, y_test

# Function to creare a simple or multiple layer perceptron
def create_mlp_model(architecture, input_shape):
    model = Sequential()
    model.add(Input((input_shape,)))

    # Add hidden layers with ReLU activation
    for neurons in architecture:
        model.add(Dense(neurons, activation='relu'))

    # Output layer with 1 neuron
    model.add(Dense(1))

    # Compile the model with AdamW optimizer and MSE loss
    model.compile(optimizer=optimizers.AdamW(learning_rate=0.001), loss='mse', metrics=['mse'])
    return model

# Plot MSE vs epochs for both models
def plot_model_curve(history, title,object):
    plt.figure(figsize=(6,4))
    plt.plot(history.history[object],'o-', label=f"Model {title}",color='#2A9D8F')
    plt.xlabel('Epoch')
    plt.ylabel(object)
    plt.title(f"Plot {object} curve for Model {title}")
    plt.legend()
    plt.show()

# Function to plot predicted vs actual values
def plot_pred_vs_actual(y_test, y_pred, title):
    plt.figure(figsize=(6,4))
    plt.scatter(y_test, y_pred, color='#F4A261', label='Scatterplot')
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='#2A9D8F', lw=2, label='Perfect prediction')  # Line of perfect predictions
    plt.title(f'Predicted vs Actual for {title}')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.show()

# Predict on the validation set and compute R2 score
def evaluate_model(model, X_val, y_val, model_name,scatterplot):
    y_pred = model.predict(X_test).flatten()
    r2 = r2_score(y_test, y_pred)
    print('\n')
    print(f'{model_name} R2 score: {r2:.4f}')
    print('\n')

    # Plot predicted vs actual values
    if scatterplot==True:
      plot_pred_vs_actual(y_test, y_pred, model_name)

    return r2

# Function for printing the data in a tabular form
def print_results(results):
    # Prepare the data for the table
    headers = ["Model", "Architecture", "Final Loss", "Final Validation MSE", "R2 Score"]
    table = []
    for i, result in enumerate(results):
        row = [
            f"Model {i+1}",                  # Model number
            result['architecture'],           # Architecture of the model
            result['final_loss'],             # Final Loss
            result['final_val_mse'],          # Final Validation MSE
            result['r2_score']                # R2 Score
        ]
        table.append(row)

    # Print the results using tabulate
    print(tabulate(table, headers=headers, floatfmt=".4f", tablefmt="fancy_grid"))


## Initial Model Architecture

In [ ]:
# Function for preparing the data and getting the needed quantities
input_data_shape,X_train,y_train,X_test,y_test= preparing_data(train,test,scaled=False)
#print(input_data_shape)


In [ ]:
# Architecture and results' list
architecture=[32]
results=[]

## Compile and Training

In [ ]:
#### Implementation and study of the model ####
print(f"Training model with architecture: {architecture}")

# Create model
model= create_mlp_model(architecture, input_shape=X_train.shape[1])

#Prints a string summary of the network.
model.summary()

# Train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, verbose=2)

# Get final loss and validation MSE
final_loss = history.history['loss'][-1]
final_val_mse = history.history['val_mse'][-1]

# Calculate R2 score
r2 = evaluate_model(model, X_test, y_test, f"Model {architecture}",scatterplot=True)

# Store results
results.append({
    'architecture': architecture,
    'final_loss': final_loss,
    'final_val_mse': final_val_mse,
    'r2_score': r2
})

In [ ]:
# Show the results
print_results(results)

## Model Performance

Studying the model performance of a neural network is useful to ensure it generalizes well to unseen data, effectively solving the intended problem without overfitting.

### Loss and MSE

In [ ]:
# Plot the loss curve
plot_model_curve(history,'Simple Perception',object='loss')

In [ ]:
# Plot the value of loss curve
plot_model_curve(history,'Simple Perception',object='val_loss')

In [ ]:
# Plot the MSE curve
plot_model_curve(history,'Simple Perception',object='mse')

In [ ]:
# Plot the value of MSE curve
plot_model_curve(history,'Simple Perception',object='val_mse')

# Multilayer Perceptron

A multilayer perceptron is beneficial because it can capture complex patterns and relationships in data through its multiple layers and non-linear activation functions, making it more powerful than single-layer networks.

## Initial MLP

In [ ]:
#### Definition of the functions ####
# Function for finding the best architecture
def find_best_architecture(results):
    best_architecture = None
    best_r2 = -float('inf')
    for result in results:
        r2 = result['r2_score']
        architecture = result['architecture']

        if r2 > best_r2:
            best_r2 = r2
            best_architecture = architecture

    # Print the architecure
    print(f"Best R2 score: {best_r2:.4f}")
    print(f"Best architecture: {best_architecture}")

    return best_r2, best_architecture

# Function for finding the worst architecture
def find_worst_architecture(results):
    worst_architecture = None
    worst_r2 = float('inf')
    for result in results:
        r2 = result['r2_score']
        architecture = result['architecture']

        if r2 < worst_r2 and r2>0:
            worst_r2 = r2
            worst_architecture = architecture

    # Print the architecure
    print(f"Worst R2 score: {worst_r2:.4f}")
    print(f"Worst architecture: {worst_architecture}")

    return worst_r2, worst_architecture


In [ ]:
#### Model architectures ####
architectures = [
    [128, 256, 512],
    [512, 256, 128],
    [256, 512, 512, 256],
    [768, 128]
]

In [ ]:
#### Train and evaluate multiple models ####
# Prepare to store results
results = []
for architecture in architectures:
    print(f"Training model with architecture: {architecture}")

    # Create model
    model= create_mlp_model(architecture, input_shape=X_train.shape[1])

    # Train model
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, verbose=2)

    # Get final loss and validation MSE
    final_loss = history.history['loss'][-1]
    final_val_mse = history.history['val_mse'][-1]

    # Calculate R2 score
    r2 = evaluate_model(model, X_test, y_test, f"Model {architecture}",scatterplot=True)

    # Store results
    results.append({
        'architecture': architecture,
        'final_loss': final_loss,
        'final_val_mse': final_val_mse,
        'r2_score': r2
    })
    # Plot of the relevant curves
    plot_model_curve(history, architecture,object='loss')
    plot_model_curve(history, architecture,object='val_loss')
    print('########################################################################')
    print('\n')


In [ ]:
# Print out results
print_results(results)

In [ ]:
# Show the best architecture
Best_R2, Best_architecture=find_best_architecture(results)

In [ ]:
# Save the results
results_normal=results

# Optimization

## Data scaling

When working with machine learning models, especially neural networks, having features
on different scales can significantly impact the model’s performance. Here’s why:
- **Convergence issues:** Features with larger scales can dominate the learning
process, causing the model to converge slowly or get stuck in suboptimal
solutions.
- **Biased importance:** The model might incorrectly assign more importance to
features with larger magnitudes, even if they’re not actually more predictive.
- **Gradient descent problems:** During optimization, features with larger scales cancause larger gradients, leading to unstable updates and potentially overshooting optimal values.
- **Activation function saturation:** For neural networks, large input values can push activation functions like ReLU or sigmoid into saturation regions, slowing down learning.

These issues can result in poor model performance, reflected in low $𝑅^2$ scores. By scaling your data, you can ensure that all features contribute more equally to the learning

In [ ]:
#### Definition of the function ####

# Function to scale the data
def scaled_train_test(df_train: pd.DataFrame, df_test: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    assert not df_train.empty, "Training DataFrame is empty"
    assert not df_test.empty, "Test DataFrame is empty"

    if not df_train.columns.equals(df_test.columns):
        raise ValueError("Training and test DataFrames must have the same columns")

    scaler = StandardScaler()
    scaled_df_train = pd.DataFrame(scaler.fit_transform(df_train), columns=df_train.columns)
    scaled_df_test = pd.DataFrame(scaler.transform(df_test), columns=df_test.columns)

    return scaled_df_train, scaled_df_test

In [ ]:
# Function for preparing the data and getting the needed quantities
input_data_shape,X_train,y_train, X_test,y_test= preparing_data(train,test,scaled=True) # Scaled=True, hence the data will be scaled
#print(input_data_shape)


In [ ]:
# Train and evaluate multiple models
results = []
for architecture in architectures:
    print(f"Training model with architecture: {architecture}, scaled case")

    # Create model
    model= create_mlp_model(architecture, input_shape=input_data_shape)

    # Train model
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, verbose=2)

    # Get final loss and validation MSE
    final_loss = history.history['loss'][-1]
    final_val_mse = history.history['val_mse'][-1]

    # Calculate R2 score
    r2 = evaluate_model(model, X_test, y_test, f"Model {architecture}",scatterplot=True)

    # Store results
    results.append({
        'architecture': architecture,
        'final_loss': final_loss,
        'final_val_mse': final_val_mse,
        'r2_score': r2
    })
    # Plot of the relevant curves
    plot_model_curve(history, architecture,object='loss')
    plot_model_curve(history, architecture,object='val_loss')
    print('########################################################################')
    print('\n')


In [ ]:
# Print out results
print_results(results)

In [ ]:
# Save the data
results_scaled=results

In [ ]:
# Compare the results wihthin the architectures
Best_R2_scaled, Best_architecture_scaled=find_best_architecture(results_scaled)
Worst_R2_scaled, Worst_architecture_scaled=find_worst_architecture(results_scaled)
print('\n')
Best_R2_normal, Best_architecture_normal=find_best_architecture(results_normal)

We can clearly notice the difference between before and after the scalar procedure.

## Helpfull layers

In [ ]:
#### Definition of the functions ####
# Function to create a model with different configurations
def create_model_with_layers(case, architecture, input_shape, dropout_rate=0.3):
    model = Sequential()

    if case == 'B':
        # FIX: Pass input_shape as a tuple
        model.add(BatchNormalization(input_shape=(input_shape,)))
        for neurons in architecture:
            model.add(Dense(neurons, activation='relu'))
    else:
        model.add(Input((input_shape,)))
        model.add(Dense(architecture[0], activation='relu'))

        if case == 'A':
            model.add(Dropout(dropout_rate))
        elif case == 'C':
            model.add(BatchNormalization())
        elif case == 'D':
            model.add(BatchNormalization())
            model.add(Dropout(dropout_rate))

    # Add the rest of the hidden layers for all cases except E
    if case != 'E':
        for neurons in architecture[1:]:
            model.add(Dense(neurons, activation='relu'))
    else:
        # In case E, apply batch normalization only after hidden layers
        for neurons in architecture:
            model.add(Dense(neurons, activation='relu'))
        model.add(BatchNormalization())

    # Output layer
    model.add(Dense(1))

    # Compile model with AdamW optimizer and MSE loss
    model.compile(optimizer=optimizers.AdamW(learning_rate=0.001), loss='mse', metrics=['mse'])

    return model

# Function to print the results for multiple cases
def print_results_multicase(results):
    # Prepare the data for the table
    headers = ["Model", "Architecture", "Final Loss", "Final Validation MSE", "R2 Score"]

    # Create separate tables for each case
    case_tables = {case: [] for case in ['A', 'B', 'C', 'D', 'E']}

    for i, result in enumerate(results):
        row = [
            f"Model {i+1}",
            result['architecture'],
            result['final_loss'],
            result['final_val_mse'],
            result['r2_score']
        ]
        # Assign the row to the correct case table
        case = result.get('case')  # Assumes 'case' is added to the results dict
        if case in case_tables:
            case_tables[case].append(row)
        else:
            print(f"Warning: Unknown case '{case}' found in results. Ignoring.")

    # Print tables for each case
    for case, table in case_tables.items():
        if table:
            print(f"## Results for Case {case}:")
            print(tabulate(table, headers=headers, floatfmt=".4f", tablefmt="latex"))
            print("\n")
        else:
            print(f"No results found for Case {case}.\n")

In [ ]:
# Experiment with different models and compare their performance
cases = ['A', 'B', 'C', 'D', 'E']
dropout_rates = [0.2, 0.3, 0.4, 0.5]


In [ ]:
# Variables unscaled
#input_data_shape,X_train,y_train, X_test,y_test= preparing_data(train,test,scaled=False) # Scaled=False, hence the data will not be scaled
#print(input_data_shape)

I decided to use scalar data because the model associated have an $R^2$ which is close to 1, hence more affected by the overfitting.

In [ ]:
# Variables scaled
input_data_shape,X_train,y_train, X_test,y_test= preparing_data(train,test,scaled=True) # Scaled=True, hence the data will be scaled
#print(input_data_shape)

In [ ]:
#### IMplementation of the functions ####

best_r2, best_architecture= find_best_architecture(results_scaled)
results_multicase = []

# Train models for each case and plot loss curves
for case in cases:
    for dropout_rate in dropout_rates if case == 'A' or case == 'D' else [None]:
        print('\n')
        print('######## Case',case,'########')
        print(f"Training case {case} with dropout rate {dropout_rate}" if dropout_rate else f"Training case {case}")
        print('\n')

        # Create model for the case
        model = create_model_with_layers(case, best_architecture, input_shape=input_data_shape, dropout_rate=dropout_rate)

        # Train model for 10 epochs
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, verbose=0)

        # Get final loss and validation MSE
        final_loss = history.history['loss'][-1]
        final_val_mse = history.history['val_mse'][-1]

        # Calculate R2 score
        r2 = evaluate_model(model, X_test, y_test, f"Model {best_architecture}",scatterplot=True)

        # Store results
        results_multicase.append({
            'case': case,  # Add the case to the dictionary
            'architecture': best_architecture,
            'final_loss': final_loss,
            'final_val_mse': final_val_mse,
            'r2_score': r2
        })

        # Plot of the relevant curves
        plot_model_curve(history, best_architecture,object='loss')
        plot_model_curve(history, best_architecture,object='val_loss')
        print('########################################################################')
        print('\n')



In [ ]:
# Print the results
print_results_multicase(results_multicase)

## Regularization

In [ ]:
# Additional library for regularizers
from keras import regularizers

In [ ]:
#### Definition of the functions ####
# Function to create a model with L1, L2 or L1_L2 regularization
def create_model_with_regularization(regularizer_type, architecture, input_shape, alpha):
    model = Sequential()

    regularizer = None
    if regularizer_type == 'L1':
        regularizer = regularizers.l1(alpha)
    elif regularizer_type == 'L2':
        regularizer = regularizers.l2(alpha)
    elif regularizer_type == 'L1_L2':
        regularizer = regularizers.l1_l2(l1=alpha, l2=alpha)

    # Input layer and first Dense layer with regularization
    model.add(Input((input_shape,)))
    model.add(Dense(architecture[0], activation='relu', kernel_regularizer=regularizer))

    # Add other hidden layers with regularization
    for neurons in architecture[1:]:
        model.add(Dense(neurons, activation='relu', kernel_regularizer=regularizer))

    # Output layer
    model.add(Dense(1))

    # Compile model with AdamW optimizer and MSE loss
    model.compile(optimizer=optimizers.AdamW(learning_rate=0.001), loss='mse', metrics=['mse'])

    return model

# Function to show the results
def print_reg_results(results):
    # Prepare the data for the table
    headers = ["Regularizer", "Alpha", "Architecture", "Final Loss", "Final Validation MSE", "R2 Score"]
    table = []
    for i, result in enumerate(results):
        row = [
            result['regularizer_type'],  # Regularizer type
            result['alpha'],            # Alpha value
            result['architecture'],       # Architecture of the model
            result['final_loss'],         # Final Loss
            result['final_val_mse'],      # Final Validation MSE
            result['r2_score']            # R2 Score
        ]
        table.append(row)

    # Print the results using tabulate
    print(tabulate(table, headers=headers, floatfmt=".4f", tablefmt="latex"))

In [ ]:
# Best architecture found previously
best_r2,best_architecture= find_best_architecture(results_scaled)

# Regularization parameter alpha
alpha = 0.001

# Experiment with L1, L2, and L1_L2 regularization and compare the results
regularization_types = ['L1', 'L2', 'L1_L2']
results_reg = []


Best R2 score: 0.8317
Best architecture: [256, 512, 512, 256]


In [ ]:
# Train models with different regularization types
for regularizer_type in regularization_types:
    print(f"Training model with {regularizer_type} regularization (alpha={alpha})")

    # Create model with specified regularization
    model = create_model_with_regularization(regularizer_type, best_architecture, input_data_shape, alpha=alpha)

    # Train model for 10 epochs
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, verbose=2)

    # Get final loss and validation MSE
    final_loss = history.history['loss'][-1]
    final_val_mse = history.history['val_mse'][-1]

    # Calculate R2 score
    r2 = evaluate_model(model, X_test, y_test, f"Model {best_architecture}",scatterplot=True)

    # Store results
    results_reg.append({
        'regularizer_type': regularizer_type,
        'alpha': alpha,
        'architecture': best_architecture,
        'final_loss': final_loss,
        'final_val_mse': final_val_mse,
        'r2_score': r2
    })

    # Plot of the relevant curves
    plot_model_curve(history,  best_architecture,object='loss')
    plot_model_curve(history,  best_architecture,object='val_loss')
    print('########################################################################')
    print('\n')


In [ ]:
# Print results
print_reg_results(results_reg)

### Epochs

In [ ]:
#### Definition of the functions ####

# Plot MSE vs epochs for both models
def plot_mse_vs_epochs(history, model_name):
    plt.plot(history.history['mse'], label='Training MSE')
    plt.plot(history.history['val_mse'], label='Validation MSE')
    plt.title(f'MSE vs Epochs for {model_name}')
    plt.xlabel('Epochs')
    plt.ylabel('Mean Squared Error (MSE)')
    plt.legend()
    plt.show()

# Function to find the second best architecture
def find_second_best_architecture(results):
    # Sort results by R2 score in descending order
    sorted_results = sorted(results, key=lambda x: x['r2_score'], reverse=True)

    # Return the second best R2 score and architecture
    if len(sorted_results) > 1:  # Check if there's at least a second best model
        second_best_r2 = sorted_results[1]['r2_score']
        second_best_architecture = sorted_results[1]['architecture']
        print(f"Second best R2 score: {second_best_r2:.4f}")
        print(f"Second best architecture: {second_best_architecture}")
        return second_best_r2, second_best_architecture
    else:
        print("There is only one model in the results, cannot determine the second best.")
        return None, None  # or raise an exception if appropriate

In [ ]:
# First and second best model
best_r2_1, best_model_1_architecture = find_best_architecture(results_scaled)
best_r2_2, best_model_2_architecture = find_second_best_architecture(results_scaled)


# List of the two best-performing architectures
architectures = [best_model_1_architecture, best_model_2_architecture]

In [ ]:
#### Implementation of the function ####

results_100 = []
# Iterate through each best architecture and train a model
for architecture in architectures:
    print(f"Training model with architecture: {architecture}, scaled case")

    # Create model with the specified architecture
    model = create_mlp_model(architecture, input_shape=input_data_shape)

    # Train the model for 100 epochs
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, verbose=2)

    # Final training loss and validation MSE
    final_loss = history.history['loss'][-1]
    final_val_mse = history.history['val_mse'][-1]

    # Calculate the R² score on the test set
    r2 = evaluate_model(model, X_test, y_test, f"Model {architecture}", scatterplot=True)

    # Store the results for this architecture
    results_100.append({
        'architecture': architecture,
        'final_loss': final_loss,
        'final_val_mse': final_val_mse,
        'r2_score': r2
    })

    # Plot MSE vs Epochs
    print('\n')
    print('########################################################################')
    plot_mse_vs_epochs(history, f"{architecture}")
    print('########################################################################')
    print('\n')



In [ ]:
# Present the final results
print_results(results_100)